# 6.5 Indexed collections of sets

In [ ]:
# install dependencies
%pip install -q amplpy

from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=['highs'],  # modules to install
    license_uuid='default',  # license to use
)  # instantiate AMPL object and register magics

Although declarations of individual sets are most common in AMPL models, sets may
also be declared in collections indexed over other sets. The principles are much the same
as for indexed collections of parameters, variables or constraints.

As an example of how indexed collections of sets can be useful, let us extend the multiperiod
production model of [Figure 4-4](../04/4_3_a_model_of_production_and_transportation.ipynb#fig-4-4) to recognize different market areas for each product.
We begin by declaring:
```
set PROD;
set AREA {PROD};
```
This says that for each member `p` of `PROD`, there is to be a set `AREA[p]`; its members
will denote the market areas in which product `p` is sold.

The market demands, expected sales revenues and amounts to be sold should be
indexed over areas as well as products and weeks:
```
param market {p in PROD, AREA[p], 1..T} >= 0;
param revenue {p in PROD, AREA[p], 1..T} >= 0;
var Sell {p in PROD, a in AREA[p], t in 1..T}
			 >= 0, <= market[p,a,t];
```
In the declarations for `market` and `revenue`, we define only the dummy index `p` that is
needed to specify the set `AREA[p]`, but for the `Sell` variables we need to define all the
dummy indices, so that they can be used to specify the upper bound `market[p,a,t]`.
This is another example in which an index defined by one phrase of an indexing expression
is used by a subsequent phrase; for each `p` from the set `PROD`, a runs over a different
set `AREA[p]`.

In the objective, the expression `revenue[p,t]*Sell[p,t]` from [Figure 4-4](../04/4_3_a_model_of_production_and_transportation.ipynb#fig-4-4)
must be replaced by a sum of revenues over all areas for product `p`:
```
maximize Total_Profit:
  sum {p in PROD, t in 1..T}
     (sum {a in AREA[p]} revenue[p,a,t]*Sell[p,a,t] -
	prodcost[p]*Make[p,t] - invcost[p]*Inv[p,t]);
```
The only other change is in the `Balance` constraints, where `Sell[p,t]` is similarly
replaced by a summation:
```
subject to Balance {p in PROD, t in 1..T}:
  Make[p,t] + Inv[p,t-1]
     = sum {a in AREA[p]} Sell[p,a,t] + Inv[p,t];
```

The complete model is shown in [Figure 6-3](../06/6_5_indexed_collections_of_sets.ipynb#fig-6-3).

<a id='fig-6-3'><center><b>Figure 6-3:</b> Multiperiod production with indexed sets (steelT3.mod).</center></a>

In [2]:
%%writefile steelT3.mod
    
set PROD;                                 # products
set AREA {PROD};                          # market areas for each product
param T > 0;                              # number of weeks
param rate {PROD} > 0;          # tons per hour produced
param inv0 {PROD} >= 0;         # initial inventory
param avail {1..T} >= 0;        # hours available in week
param market {p in PROD, AREA[p], 1..T} >= 0;
					  # limit on tons sold in week
param prodcost {PROD} >= 0;     # cost per ton produced
param invcost {PROD} >= 0;      # carrying cost/ton of inventory
param revenue {p in PROD, AREA[p], 1..T} >= 0;
				# revenue per ton sold
var Make {PROD,1..T} >= 0;      # tons produced
var Inv {PROD,0..T} >= 0;       # tons inventoried
var Sell {p in PROD, a in AREA[p], t in 1..T}   # tons sold
		    >= 0, <= market[p,a,t];
maximize Total_Profit:
	sum {p in PROD, t in 1..T}
      (sum {a in AREA[p]} revenue[p,a,t]*Sell[p,a,t] -
	 prodcost[p]*Make[p,t] - invcost[p]*Inv[p,t]);
			     # Total revenue less costs for all products in all weeks
subject to Time {t in 1..T}:
	sum {p in PROD} (1/rate[p]) * Make[p,t] <= avail[t];
			     # Total of hours used by all products
			     # may not exceed hours available, in each week
subject to Init_Inv {p in PROD}: Inv[p,0] = inv0[p];
			     # Initial inventory must equal given value
subject to Balance {p in PROD, t in 1..T}:
	Make[p,t] + Inv[p,t-1]
      = sum {a in AREA[p]} Sell[p,a,t] + Inv[p,t];
			     # Tons produced and taken from inventory
			     # must equal tons sold and put into inventory

Writing steelT3.mod


In the data for this model, each set within the indexed collection `AREA` is specified
like an ordinary set:

<a id='fig-6-4'><center><b>Figure 6-4:</b> Data for multiperiod production with indexed sets.</center></a>

In [12]:
import pandas as pd

# Define time periods
T = 4
weeks = list(range(1, T + 1))

# Sets
PROD = ["bands", "coils"]
AREA = {
    "bands": ["east", "north"],
    "coils": ["east", "west", "export"]
}

avail = {1: 40, 2: 40, 3: 32, 4: 40}
rate = {"bands": 200, "coils": 140}
inv0 = {"bands": 10, "coils": 0}
prodcost = {"bands": 10, "coils": 11}
invcost = {"bands": 2.5, "coils": 3}

# Revenue data
revenue_values = [
    [25.0, 26.0, 27.0, 27.0],  # bands-east
    [26.5, 27.5, 28.0, 28.5],  # bands-north
    [30, 35, 37, 39],          # coils-east
    [29, 32, 33, 35],          # coils-west
    [25, 25, 25, 28]           # coils-export
]
# revenue_index is basically: (bands, east), (bands, north),
#(coils, east), (coils, west), and (coils, export)
revenue_index = [(p, a) for p in PROD for a in AREA[p]]
revenue_df = pd.DataFrame(revenue_values, index=pd.MultiIndex.from_tuples(revenue_index, names=["p", "a"]), columns=weeks)

# Market data, shares same index as revenue
market_values = [
    [2000, 2000, 1500, 2000],  # bands-east
    [4000, 4000, 2500, 4500],  # bands-north
    [1000, 800, 1000, 1100],   # coils-east
    [2000, 1200, 2000, 2300],  # coils-west
    [1000, 500, 500, 800]      # coils-export
]
market_df = pd.DataFrame(market_values, index=pd.MultiIndex.from_tuples(revenue_index, names=["p", "a"]), columns=weeks)

In [13]:
# Instantiate AMPL object
ampl = AMPL()

# Read model file
ampl.read("steelT3.mod")

# Load data
ampl.set["PROD"] = PROD
ampl.set["AREA"] = AREA
ampl.param["T"] = T
ampl.param["avail"] = avail
ampl.param["rate"] = rate
ampl.param["inv0"] = inv0
ampl.param["prodcost"] = prodcost
ampl.param["invcost"] = invcost
ampl.param["revenue"].set_values(revenue_df)
ampl.param["market"].set_values(market_df)

# Solve
ampl.solve(solver='highs')
print("Objective:", ampl.get_objective("Total_Profit").value())


HiGHS 1.11.0: optimal solution; objective 514521.7143
18 simplex iterations
0 barrier iterations
Objective: 514521.7142857143


The parameters revenue and market are now indexed over three sets, so their data
values are specified in a series of tables. Since the indexing is over a different
set `AREA[p]` for each product `p`, the values are most conveniently arranged as one table for
each product, as shown in [Figure 6-4](../06/6_5_indexed_collections_of_sets.ipynb#fig-6-4). (Chapter 9 xTODO explains the general rules behind this
arrangement.)
We could instead have written this model with a set `PRODAREA` of pairs, such that
product `p` will be sold in area `a` if and only if `(p,a)` is a member of `PRODAREA`. Our
formulation in terms of `PROD` and `AREA[p]` seems preferable, however, because it
emphasizes the hierarchical relationship between products and areas. Although the model
must refer in many places to the set of all areas selling one product, it never refers to the
set of all products sold in one area.

<a id='fig-6-5'><center><b>Figure 6-5:</b> Multicommodity transportation with indexed sets (multic.mod).</center></a>

In [ ]:
%%writefile multic.mod

set ORIG;                       # origins
set DEST;                       # destinations
set PROD;                       # products
set orig {PROD} within ORIG;
set dest {PROD} within DEST;
set links {p in PROD} = orig[p] cross dest[p];
param supply {p in PROD, orig[p]} >= 0; # available at origins
param demand {p in PROD, dest[p]} >= 0; # required at destinations
check {p in PROD}: sum {i in orig[p]} supply[p,i]
			 = sum {j in dest[p]} demand[p,j];
param limit {ORIG,DEST} >= 0;
param cost {p in PROD, links[p]} >= 0;  # shipment costs per unit
var Trans {p in PROD, links[p]} >= 0;   # units to be shipped
minimize Total_Cost:
    sum {p in PROD, (i,j) in links[p]} cost[p,i,j] * Trans[p,i,j];
subject to Supply {p in PROD, i in orig[p]}:
    sum {j in dest[p]} Trans[p,i,j] = supply[p,i];
subject to Demand {p in PROD, j in dest[p]}:
    sum {i in orig[p]} Trans[p,i,j] = demand[p,j];
subject to Multi {i in ORIG, j in DEST}:
    sum {p in PROD: (i,j) in links[p]} Trans[p,i,j] <= limit[i,j];


As a contrasting example, we can consider how the multicommodity transportation
model might use indexed collections of sets. As shown in [Figure 6-5](../06/6_5_indexed_collections_of_sets.ipynb#fig-6-5), for each product
we define a set of origins where that product is supplied, a set of destinations where the
product is demanded, and a set of links that represent possible shipments of the product:
```
set orig {PROD} within ORIG;
set dest {PROD} within DEST;
set links {p in PROD} = orig[p] cross dest[p];
```
The declaration of links demonstrates that it is possible to have an indexed collection
of compound sets, and that an indexed collection may be defined through set operations
from other indexed collections. In addition to the operations previously mentioned, there
are iterated union and intersection operators that apply to sets in the same way that an
iterated sum applies to numbers. For example, the expressions
```
union {p in PROD} orig[p]
inter {p in PROD} orig[p]
```
represent the subset of origins that supply at least one product, and the subset of origins
that supply all products.

The hierarchical relationship based on products that was observed in [Figure 6-3](../06/6_5_indexed_collections_of_sets.ipynb#fig-6-3) is
seen in most of [Figure 6-5](../06/6_5_indexed_collections_of_sets.ipynb#fig-6-5) as well. The model repeatedly deals with the sets of all origins,
destinations, and links associated with a particular product. The only exception
comes in the last constraint, where the summation must be over all products shipped via a
particular link:
```
subject to Multi {i in ORIG, j in DEST}:
      sum {p in PROD: (i,j) in links[p]} Trans[p,i,j] <= limit[i,j];
```
Here it is necessary, following sum, to use a somewhat awkward indexing expression to
describe a set that does not match the hierarchical organization.

In general, almost any model that can be written with indexed collections of sets can
also be written with sets of tuples. As our examples suggest, indexed collections are most
suitable for entities such as products and areas that have a hierarchical relationship. Sets
of tuples are preferable, on the other hand, in dealing with entities like origins and destinations
that are related symmetrically.
